In [1]:
import pandas as pd
from pathlib import Path

In [2]:
DATA_DIR = Path("../datasets")

df_ventas = pd.read_csv(DATA_DIR / 'ventas_completo.csv')
df_clasificados = pd.read_csv(DATA_DIR / 'productos_clasificados.csv')


df_ventas.head(3)

,fecha,product_name,category_off,perecedero,ventas,precio,en_promocion,en_temporada,temp_inicio_mes,temp_fin_mes
0,2022-12-01,Néctar de durazno 1 L,juice-box,0,10,33.72,0,0,7.0,9.0
1,2022-12-02,Néctar de durazno 1 L,juice-box,0,5,33.66,0,0,7.0,9.0
2,2022-12-03,Néctar de durazno 1 L,juice-box,0,7,31.48,0,0,7.0,9.0


In [3]:
df_clasificados = df_clasificados[['product_name', 'perecedero']]
df_clasificados.head(3)

,product_name,perecedero
0,Néctar de durazno 1 L,0
1,Sardinas en salsa 155 g,0
2,Queso gouda 300 g,1


In [16]:
df_ventas_cp = df_ventas[['fecha', 'product_name', 'category_off', 'ventas', 'precio']]
df_ventas_cp['fecha'] = pd.to_datetime(df_ventas_cp['fecha'])
df_ventas_cp = df_ventas_cp.sort_values(by='fecha', ascending=True)
df_ventas_cp.to_csv(DATA_DIR / 'ventas.csv', index=False)
df_ventas_cp.head(3)

/tmp/ipykernel_112626/3164664437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas_cp['fecha'] = pd.to_datetime(df_ventas_cp['fecha'])


,fecha,product_name,category_off,ventas,precio
0,2022-12-01,Néctar de durazno 1 L,juice-box,10,33.72
20824,2022-12-01,Arroz blanco 500 g,rice-white-dry,3,45.84
278384,2022-12-01,Betabel,fruits-vegetables,33,30.22


In [17]:
df_ventas_perecederos = pd.merge(df_ventas_cp, df_clasificados, how="left", on= "product_name")
df_ventas_perecederos.loc[df_ventas_perecederos['category_off'] == 'fruits-vegetables', 'perecedero'] = 1
df_ventas_perecederos[df_ventas_perecederos['perecedero'].isna()]

,fecha,product_name,category_off,ventas,precio,perecedero


In [18]:
# df_ventas_perecederos[['en_temporada', 'temp_inicio_mes','temp_fin_mes']]
df_ventas_perecederos['en_temporada'] = 0
df_ventas_perecederos['temp_inicio_mes'] = 1 
df_ventas_perecederos['temp_fin_mes'] = 12

In [19]:
df_frutas_verduras_temp = pd.read_csv(DATA_DIR / 'frutas-verduras_temporada_normalizado.csv')
df_frutas_verduras_temp.head(3)

,Unnamed: 0,nombre,tipo,temporada_pico,todo_el_anio,temporada_pico_inicio,temporada_pico_fin
0,0,Aguacate,fruits-vegetables,Dic-May (alta); disponible todo el año,True,12.0,5.0
1,1,Ciruela,fruits-vegetables,Jul–Ago,False,7.0,8.0
2,2,Durazno,fruits-vegetables,Jul–Sep,False,7.0,9.0


In [20]:
df_ventas_perecederos[df_ventas_perecederos['product_name'].str.contains('aguacate', case=False, na=False)]


,fecha,product_name,category_off,ventas,precio,perecedero,en_temporada,temp_inicio_mes,temp_fin_mes
57,2022-12-01,Aguacate criollo,avocado,19,62.76,1.0,0,1,12
58,2022-12-01,Aguacate criollo,avocado,19,62.76,1.0,0,1,12
59,2022-12-01,Aguacate criollo,avocado,19,62.76,1.0,0,1,12
126,2022-12-01,Aguacate criollo,avocado,18,64.17,1.0,0,1,12
127,2022-12-01,Aguacate criollo,avocado,18,64.17,1.0,0,1,12
...,...,...,...,...,...,...,...,...,...
840489,2025-11-30,Aguacate Hass,avocado,18,40.88,1.0,0,1,12
840490,2025-11-30,Aguacate Hass,avocado,18,40.88,1.0,0,1,12
840507,2025-11-30,Aguacate Hass,avocado,24,39.73,1.0,0,1,12
840508,2025-11-30,Aguacate Hass,avocado,24,39.73,1.0,0,1,12


In [21]:
import unicodedata
import pandas as pd

def normalizar(texto):
    if pd.isna(texto):
        return ""
    texto = str(texto).lower().strip()
    texto = unicodedata.normalize("NFD", texto)
    texto = "".join(c for c in texto if unicodedata.category(c) != "Mn")  # quita acentos
    return texto

# Crear columnas normalizadas
df_frutas_verduras_temp['nombre'] = df_frutas_verduras_temp['nombre'].apply(normalizar)
df_ventas_perecederos['product_name'] = df_ventas_perecederos['product_name'].apply(normalizar)


In [22]:
for _, row in df_frutas_verduras_temp.iterrows():
    nombre = row['nombre']  # ej. "aguacate"

    # Buscar los product_name que contengan ese nombre
    mask = df_ventas_perecederos['product_name'].str.contains(nombre, na=False)

    # Asignar inicio y fin de temporada
    df_ventas_perecederos.loc[mask, 'temp_inicio_mes'] = row['temporada_pico_inicio']
    df_ventas_perecederos.loc[mask, 'temp_fin_mes']    = row['temporada_pico_fin']
    

/tmp/ipykernel_112626/1465665850.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df_ventas_perecederos['product_name'].str.contains(nombre, na=False)
/tmp/ipykernel_112626/1465665850.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df_ventas_perecederos['product_name'].str.contains(nombre, na=False)
/tmp/ipykernel_112626/1465665850.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df_ventas_perecederos['product_name'].str.contains(nombre, na=False)
/tmp/ipykernel_112626/1465665850.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df_ventas_perecederos['product_name'].str.contains(nombre, na=False)


In [23]:
df_ventas_perecederos

,fecha,product_name,category_off,ventas,precio,perecedero,en_temporada,temp_inicio_mes,temp_fin_mes
0,2022-12-01,nectar de durazno 1 l,juice-box,10,33.72,0.0,0,7.0,9.0
1,2022-12-01,nectar de durazno 1 l,juice-box,10,33.72,0.0,0,7.0,9.0
2,2022-12-01,arroz blanco 500 g,rice-white-dry,3,45.84,0.0,0,1.0,12.0
3,2022-12-01,arroz blanco 500 g,rice-white-dry,3,45.84,0.0,0,1.0,12.0
4,2022-12-01,arroz blanco 500 g,rice-white-dry,3,45.84,0.0,0,1.0,12.0
...,...,...,...,...,...,...,...,...,...
840627,2025-11-30,cafe molido 500 g,coffee-ground,1,54.91,0.0,0,1.0,12.0
840628,2025-11-30,cafe molido 500 g,coffee-ground,1,54.91,0.0,0,1.0,12.0
840629,2025-11-30,queso fresco 250 g,cheese-fresh,17,55.21,1.0,0,1.0,12.0
840630,2025-11-30,queso fresco 250 g,cheese-fresh,17,55.21,1.0,0,1.0,12.0


In [ ]:
mes = df_ventas_perecederos['fecha'].dt.month
ini = df_ventas_perecederos['temp_inicio_mes']
fin = df_ventas_perecederos['temp_fin_mes']

# Caso A: temporada normal (inicio <= fin)
caso_1 = (ini <= fin) & (mes >= ini) & (mes <= fin)

# Caso B: temporada que cruza año (inicio > fin)
# Ej: 12–5 → meses validos son 12,1,2,3,4,5
caso_2 = (ini > fin) & ((mes >= ini) | (mes <= fin))

df_ventas_perecederos['en_temporada'] = 0
df_ventas_perecederos.loc[caso_1 | caso_2, 'en_temporada'] = 1


In [29]:
df_ventas_perecederos['temp_inicio_mes'] = df_ventas_perecederos['temp_inicio_mes'].fillna(1)
df_ventas_perecederos['temp_fin_mes'] = df_ventas_perecederos['temp_fin_mes'].fillna(12)
